In [144]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [38]:
# import train and test dataset
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
full_data = [train, test]

In [41]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Data Exploration

In [48]:
# exploratory data analysis
print('Table information: ', train.info())
print('Table desc: ', train.describe())
print('Is there any null values? \n', train.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
 12  FamilySize   891 non-null    int64  
dtypes: float64(2), int64(6), object(5)
memory usage: 90.6+ KB
Table information:  None
Table desc:         PassengerId    Survived      Pclass         Age       SibSp  \
count   891.000000  891.000000  891.000000  714.000000  891.000000   
mean    446.000000    0.

In [46]:
print('Gender distribution:\n',train['Sex'].value_counts()); print('\n')
print ('Mean of gender survived:\n', train[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean()); print('\n')
print ('Mean of Pclass survived:\n', train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean())

Gender distribution:
 male      577
female    314
Name: Sex, dtype: int64


Mean of gender survived:
       Sex  Survived
0  female  0.742038
1    male  0.188908


Mean of Pclass survived:
    Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363


In [42]:
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
print (train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean())

   FamilySize  Survived
0           1  0.303538
1           2  0.552795
2           3  0.578431
3           4  0.724138
4           5  0.200000
5           6  0.136364
6           7  0.333333
7           8  0.000000
8          11  0.000000


## Data Cleaning & Modelling

In [61]:
# Label Encoder to convert categorical gender to numerical (1 and 0)
le = preprocessing.LabelEncoder()
train['Sex'] = le.fit_transform(train['Sex'].astype(str))
train['Embarked'] = le.fit_transform(train['Embarked'].astype(str))

In [93]:
# Missing values
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
imp_mean.fit(train['Age'].values.reshape(-1, 1))
train['Age'] = imp_mean.transform(train['Age'].values.reshape(-1, 1))

In [230]:
# Choosing class and variables (X for variables, y for class)
val_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Survived', 'FamilySize']
X = train.drop(val_drop, axis=1)
y = train['Survived']

In [231]:
# Standarization
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [232]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [233]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(712, 7)
(712,)
(179, 7)
(179,)


In [234]:
# using PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pca_attributes_train = pca.fit_transform(X_train)
pca_attribute_test = pca.fit_transform(X_test)
pca.explained_variance_ratio_

array([0.30007229, 0.23716759, 0.14675717])

### Decision Tree

In [235]:
# building decision tree model
decision_tree = tree.DecisionTreeClassifier(criterion='entropy')

In [236]:
# Without PCA
train_mode = decision_tree.fit(X_train, y_train)
train_mode.score(X_test, y_test)

0.8100558659217877

In [222]:
# With PCA
train_mode_pca = decision_tree.fit(pca_attributes_train, y_train)
train_mode_pca.score(pca_attribute_test, y_test)

0.6983240223463687

### Random Forest (With GridSearch)

In [237]:
# building random forest model
clf = RandomForestClassifier(max_depth=5, criterion='gini', random_state=0)

In [152]:
# Without PCA
clf_train = clf.fit(X_train, y_train)
clf_train.score(X_test, y_test)

0.8044692737430168

In [141]:
# With PCA
clf_train_pca = clf.fit(pca_attributes_train, y_train)
clf_train.score(pca_attribute_test, y_test)

0.6871508379888268

In [149]:
clf_model = RandomForestClassifier()

parameters = {
    'criterion': ['entropy', 'gini'],
    'max_depth':     [3, 5, 10]
}
grid_search = GridSearchCV(clf_model, parameters)

In [150]:
grid_search.fit(X_train,y_train)
print(grid_search.best_params_)

{'criterion': 'gini', 'max_depth': 5}


### Predict Using Decision Tree

In [238]:
dct_model = decision_tree.fit(X, y)

In [211]:
test = pd.read_csv('test.csv')
test = test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

In [223]:
test['Sex'] = le.fit_transform(test['Sex'].astype(str))
test['Embarked'] = le.fit_transform(test['Embarked'].astype(str))

In [224]:
imp_mean.fit(test['Age'].values.reshape(-1, 1))
test['Age'] = imp_mean.transform(test['Age'].values.reshape(-1, 1))
test['Fare'] = imp_mean.transform(test['Fare'].values.reshape(-1, 1))
test = scaler.fit_transform(test)

In [240]:
test_predict = dct_model.predict(test)

In [243]:
from sklearn.metrics import accuracy_score

y_test_dataset = pd.read_csv("gender_submission.csv")
y_test = y_test_dataset.iloc[:, 1].values
print('accuracy = ' + str(accuracy_score(y_test, test_predict)))

accuracy = 0.7679425837320574


In [245]:
test_predict

array([0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,

In [253]:
result = []
for i in range(892, 1310):
    row = [i, test_predict[i - 892]]
    result.append(row)
import csv
with open('titanic_result_decision_tree.csv', "w+") as csv_file:
    csvWriter = csv.writer(csv_file, delimiter = ',')
    csvWriter.writerow(['PassengerID', 'Survived'])
    csvWriter.writerows(result)


In [247]:
result = pd.DataFrame(data=result[;])

[[892, 0],
 [893, 0],
 [894, 0],
 [895, 0],
 [896, 1],
 [897, 1],
 [898, 1],
 [899, 1],
 [900, 0],
 [901, 0],
 [902, 0],
 [903, 0],
 [904, 1],
 [905, 0],
 [906, 1],
 [907, 1],
 [908, 0],
 [909, 1],
 [910, 1],
 [911, 0],
 [912, 1],
 [913, 1],
 [914, 0],
 [915, 0],
 [916, 1],
 [917, 0],
 [918, 1],
 [919, 1],
 [920, 0],
 [921, 1],
 [922, 0],
 [923, 0],
 [924, 1],
 [925, 0],
 [926, 0],
 [927, 1],
 [928, 0],
 [929, 0],
 [930, 0],
 [931, 0],
 [932, 0],
 [933, 0],
 [934, 0],
 [935, 1],
 [936, 1],
 [937, 0],
 [938, 0],
 [939, 0],
 [940, 1],
 [941, 1],
 [942, 1],
 [943, 1],
 [944, 1],
 [945, 0],
 [946, 0],
 [947, 0],
 [948, 0],
 [949, 0],
 [950, 0],
 [951, 1],
 [952, 0],
 [953, 1],
 [954, 0],
 [955, 0],
 [956, 1],
 [957, 1],
 [958, 0],
 [959, 0],
 [960, 0],
 [961, 1],
 [962, 1],
 [963, 0],
 [964, 0],
 [965, 0],
 [966, 1],
 [967, 0],
 [968, 0],
 [969, 0],
 [970, 1],
 [971, 1],
 [972, 1],
 [973, 0],
 [974, 0],
 [975, 0],
 [976, 1],
 [977, 1],
 [978, 0],
 [979, 1],
 [980, 1],
 [981, 1],
 [982, 1],

SyntaxError: invalid syntax (<ipython-input-251-a2051f021116>, line 1)